In [2]:
#import related libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train = tf.image.resize(x_train, (75, 75))  
x_test = tf.image.resize(x_test, (75, 75))
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)
x_train, x_test = x_train / 255.0, x_test / 255.0


In [4]:
base_model = InceptionV3(input_shape=(75, 75, 3), include_top=False, weights='imagenet')

In [5]:
for layer in base_model.layers:
    layer.trainable = False


In [6]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x) 
x = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(base_model.input, x)


In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 133s 71ms/step - loss: 0.5152 - accuracy: 0.8164 - val_loss: 0.4304 - val_accuracy: 0.8411
Epoch 2/10
1875/1875 [==============================] - 108s 57ms/step - loss: 0.3851 - accuracy: 0.8580 - val_loss: 0.4074 - val_accuracy: 0.8538
Epoch 3/10
1875/1875 [==============================] - 104s 55ms/step - loss: 0.3311 - accuracy: 0.8784 - val_loss: 0.4320 - val_accuracy: 0.8512
Epoch 4/10
1875/1875 [==============================] - 109s 58ms/step - loss: 0.2845 - accuracy: 0.8945 - val_loss: 0.4420 - val_accuracy: 0.8509
Epoch 5/10
1875/1875 [==============================] - 106s 57ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4512 - val_accuracy: 0.8557
Epoch 6/10
1875/1875 [==============================] - 107s 57ms/step - loss: 0.2041 - accuracy: 0.9236 - val_loss: 0.5105 - val_accuracy: 0.8480
Epoch 7/10
1875/1875 [==============================] - 103s 55ms/step - loss: 0.1718 - accuracy: 0.9365 - val_loss: 0

In [9]:
model.evaluate(x_test,  y_test, verbose=2)


313/313 - 15s - loss: 0.6735 - accuracy: 0.8462 - 15s/epoch - 47ms/step


[0.6735058426856995, 0.8461999893188477]

In [13]:
#visualize the model training accuracy and loss

plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


KeyError: 'accuracy'

In [15]:
from sklearn.metrics import confusion_matrix
#confusion matrix
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)


313/313 [==============================] - 15s 48ms/step
[[737   5  34  46  22   1 143   0  10   2]
 [  0 964   4  20   5   0   6   0   1   0]
 [  6   5 721  12 156   0  96   0   4   0]
 [ 37  11  16 806  60   1  64   0   4   1]
 [  2   1  38  32 839   1  78   0   9   0]
 [  2   1   0   0   0 934   1  46   4  12]
 [102   2  45  35 170   0 634   1  11   0]
 [  0   0   0   0   0  37   0 917   0  46]
 [  5   1  14   1   6   5   8   1 954   5]
 [  2   1   0   0   0  11   0  29   1 956]]
